In [85]:
import pandas
data_training = pandas.read_csv('T2.csv') 
data_testing = pandas.read_csv('validsplit21.csv') 

In [86]:
L1 = data_training['lev_pref1'].values
L2 = data_training['lev_pref2'].values
M1 = data_testing['lev_pref1'].values
M2 = data_testing['lev_pref2'].values

In [87]:
Train_SET1 = []
Test_SET1 = []
for i in range(len(L)):
    if L1[i]>=5 and L[i]>=5.5:
        Train_SET1.append(i)
for i in range(len(M)):
    if M1[i]>7 and M[i]>4.5:
        Test_SET1.append(i)
import numpy as np
x = set(list(np.arange(len(L))))
Train_SET2 = x - set(Train_SET1) 
data_training_set2 = data_training.drop(data_training.index[Train_SET1])
data_training_set1 = data_training.drop(data_training.index[list(Train_SET2)])

x = set(list(np.arange(len(M))))
Test_SET2 = x - set(Test_SET1) 
data_testing_set2 = data_testing.drop(data_testing.index[Test_SET1])
data_testing_set1 = data_testing.drop(data_testing.index[list(Test_SET2)])


In [88]:
new_features_train = data_training.columns[3:].drop(['Cit_pref1','actual_vote'])
new_features_test = data_training.columns[3:].drop(['Cit_pref1','actual_vote'])

In [89]:
new_features_test= ['Ebony','Odyssey','Tokugawa','mvar32','mvar33','Cosmos','Centaur','city_change','ZIP_pref1','ZIP_pref2','mvar34','mvar35','mvar29','lev_pref1','lev_pref2','ZIP_lev_pref1','ZIP_lev_pref2','mvar27','mvar28','mvar31']
Feature = data_testing_set1.columns[3:28]
new_features_train= ['Ebony','Odyssey','Tokugawa','Cosmos','Centaur','mvar32','mvar33','city_change','ZIP_pref1','ZIP_pref2','mvar34','mvar35','mvar29','lev_pref1','lev_pref2','ZIP_lev_pref1','ZIP_lev_pref2','mvar27','mvar28','mvar31']
new_features_test = new_features_test +  list(Feature)
new_features_train = new_features_train + list(Feature)
Y_SET1 = data_training_set1['actual_vote'].values
Y_SET2 = data_training_set2['actual_vote'].values

In [90]:
from sklearn.preprocessing import MinMaxScaler
MinMax = MinMaxScaler()
import warnings
warnings.filterwarnings('ignore')

for df in [data_testing_set1,data_testing_set2,data_training_set1,data_training_set2]:
            for i in Feature:
                df[i] = df[i].astype(str)
            df = df[new_features_train][:]
            df['mvar27'] = df['mvar27'].apply(str)
            #df['lev_pref1'] = MinMax.fit_transform(df['lev_pref1'].values)
            #df['ZIP_lev_pref1'] = MinMax.fit_transform(df['ZIP_lev_pref1'].values)
            #df['ZIP_lev_pref2'] = MinMax.fit_transform(df['ZIP_lev_pref2'].values)
            #df['lev_pref2'] = MinMax.fit_transform(df['lev_pref2'].values)
            df['mvar35'] = MinMax.fit_transform(df['mvar35'].values)
            df['mvar34'] = MinMax.fit_transform(df['mvar34'].values)
            df['mvar28'] = MinMax.fit_transform(df['mvar28'].values)
            df['mvar31'] = MinMax.fit_transform(df['mvar31'].values)
            df['mvar29'] = MinMax.fit_transform(df['mvar29'].values)

In [91]:
#ENCODING
data_training_1 = data_training_set1[new_features_test][:]
data_testing_1 = data_testing_set1[new_features_test][:]
X_train = data_training_1.to_dict('records')
X_test = data_testing_1.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te
#One Hot Encoding 
from sklearn.feature_extraction import DictVectorizer 
enc = DictVectorizer(sparse = True)
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train_set1 =X_encoded_total[:len(X_tr)]
X_encoded_test_set1 =X_encoded_total[len(X_tr):]

In [92]:
#ENCODING
data_training_2 = data_training_set2[new_features_test][:]
data_testing_2 = data_testing_set2[new_features_test][:]
X_train = data_training_2.to_dict('records')
X_test = data_testing_2.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te
#One Hot Encoding 
from sklearn.feature_extraction import DictVectorizer 
enc = DictVectorizer(sparse = True)
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train_set2 =X_encoded_total[:len(X_tr)]
X_encoded_test_set2 =X_encoded_total[len(X_tr):]

In [93]:
from sklearn.linear_model import LogisticRegression
clf_SET1 = LogisticRegression(solver = 'newton-cg',fit_intercept = False)
clf_SET2 = LogisticRegression(solver  ='newton-cg',fit_intercept = False,C = 0.01)
clf_SET1.fit(X_encoded_train_set1,Y_SET1)
clf_SET2.fit(X_encoded_train_set2,Y_SET2)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

In [94]:
sol_SET1 =clf_SET1.predict(X_encoded_test_set1.toarray())
sol_SET2 =clf_SET2.predict(X_encoded_test_set2.toarray())

In [95]:
data_testing_set2['Predicted'] = sol_SET2
data_testing_set1['Predicted'] = sol_SET1
DF1 = data_testing_set1[['citizen_id','Predicted']]
DF2 = data_testing_set2[['citizen_id','Predicted']]


In [99]:
import pandas as pd
frames = [DF1, DF2]
result = pd.concat(frames)

In [100]:
result.to_csv('F8.csv', index=False, header=False)

In [88]:
data_testing['Predicted'] = result['Predicted']

In [89]:
df1 = data_testing
History_vote = []
for i in range(len(df1)):
    if df1['Predicted'].values[i] == df1['Predicted'].values[i]:
        if df1['party_voted_past'].values[i] == df1['Predicted'].values[i]:
             History_vote.append(1)
        else:
             History_vote.append(0)

In [92]:
sum(History_vote)

17830

In [78]:
# Split the dataset in two equal parts
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_train_set1, Y_SET1, test_size=0.75, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'multi_class': ['ovr','multinomial']}]

scores = ['precision']
clf10 = LogisticRegression()
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf_SET1, tuned_parameters, cv=5, scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_estimator_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision
()
Best parameters set found on development set:
()
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)
()
Grid scores on development set:
()
0.607 (+/-0.006) for {'multi_class': 'ovr'}
0.597 (+/-0.006) for {'multi_class': 'multinomial'}
()
Detailed classification report:
()
The model is trained on the full development set.
The scores are computed on the full evaluation set.
()
             precision    recall  f1-score   support

    Centaur       0.62      0.64      0.63      5794
     Cosmos       0.64      0.65      0.65      5970
      Ebony       0.57      0.51      0.54      4549
    Odyssey       0.61      0.64      0.63      4939
   Tokugawa       0.62      0.62      0.62      3809

avg / total       0.61      0.62      0.61  

In [11]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...


H2OStartupError: Cannot find Java. Please install the latest JDK from
http://www.oracle.com/technetwork/java/javase/downloads/index.html

In [84]:
79150 +590300

669450